In [3]:
import numpy as np
import torch
import time

In [4]:
# sampling from independent normals using parameterization trick
def sample(num_concepts, num_latent_concepts, batch_size):
    mu_hat = np.random.random(num_concepts).reshape(num_concepts, 1)*2 # (n_concepts,  1)
    true_sigma_hat = np.random.random(num_concepts).reshape(num_concepts, 1)*5 # (n_concepts,  1)
    c_hat = np.random.standard_normal(batch_size*num_concepts).reshape(batch_size, num_concepts, 1) * true_sigma_hat**0.5 + mu_hat # (batch_size, n_concepts,  1)

    mu_tilde = np.random.random(num_latent_concepts).reshape(num_latent_concepts, 1)*5 # (n_latent,  1)
    true_sigma_tilde = np.random.random(num_latent_concepts).reshape(num_latent_concepts, 1)*10 # (n_latent,  1)
    c_tilde = np.random.standard_normal(batch_size*num_latent_concepts).reshape(batch_size, num_latent_concepts, 1) * true_sigma_tilde**0.5 + mu_tilde # (batch_size, n_latent,  1)
    # print(list(true_sigma_hat.squeeze())+list(true_sigma_tilde.squeeze()))
    return c_hat, c_tilde

# c_hat_mean = np.mean(c_hat, axis=0) # (n_concepts, 1)
# c_tilde_mean = np.mean(c_tilde, axis=0) # (n_latent, 1)
# print(c_hat.shape, c_tilde.shape)

# sampling from independent normals using parameterization trick
def sample_c(num_concepts, batch_size):
    mu_hat = np.random.random(num_concepts).reshape(num_concepts)*2 # (n_concepts,  1)
    true_sigma_hat = np.random.random(num_concepts*num_concepts).reshape(num_concepts, num_concepts)
    true_sigma_hat = np.matmul(true_sigma_hat, true_sigma_hat.T)*5 # (n_concepts,  1)
    c_hat = np.random.multivariate_normal(mu_hat, true_sigma_hat, batch_size)
    return c_hat, mu_hat, true_sigma_hat

# c_hat_mean = np.mean(c_hat, axis=0) # (n_concepts, 1)
# c_tilde_mean = np.mean(c_tilde, axis=0) # (n_latent, 1)
# print(c_hat.shape, c_tilde.shape)

In [5]:
def get_corr_to_std(batch_size, std=1, mean=0, samples=100):
    min_corr = -1
    max_corr = 1
    step = 0.01
    scale_log = 2
    scale = 10**scale_log
    corr_to_std = {}

    for corr in range(int(scale*min_corr), int((max_corr+step)*scale), int(step*scale)):
        corr /= scale
        cov_matrix = np.ones((2,2))
        np.fill_diagonal(cov_matrix, std**2)
        cov_matrix[0,1] = corr*std**2
        cov_matrix[1,0] = corr*std**2
        z = np.random.multivariate_normal(np.ones(2)*mean, cov_matrix, (samples, batch_size))
        corr_z = []
        for sample in z:
            corr_z.append(np.corrcoef(sample, rowvar=False)[0,1])
        std_corr_z = np.std(corr_z)
        corr_to_std[int(corr*scale)] = std_corr_z

        def get_std(correlation): # interpolation
            return corr_to_std[int(round(correlation, scale_log)*scale)]
            # if correlation == 1:
            #     return corr_to_std[correlation]
            # if correlation < step:
            #     return corr_to_std[0]*correlation*100 + corr_to_std[int(step*scale)]*(1-correlation*100)
            # corr_truncate = int(correlation * scale) / scale
            # alpha = int(correlation*scale*100) % int(corr_truncate*scale*100) / 100 # 0.1234 -> 0.34   (1234%1200)
            # return corr_to_std[int(corr_truncate*scale)]*alpha + corr_to_std[int((corr_truncate+step)*scale)]*(1-alpha)
    return get_std

In [27]:
def get_cov(c, get_std, delta=1):
    p_hat = np.corrcoef(c, rowvar=False) # 256+128, 256+128

    # calculate S_p
    start = time.time()
    S_p = np.sum(get_std(p_hat)**2)**0.5
    print('time to convert:', time.time()-start)

    # estimate gamma
    for gamma in range(2, 10, 2):
        p_hat_gamma = p_hat**gamma * p_hat
        # print('gamma:', gamma, np.linalg.norm(p_hat - p_hat_gamma), delta * S_p)
        if np.linalg.norm(p_hat - p_hat_gamma) >= delta * S_p:
            gamma_star = gamma
            break

    # estimate alpha
    scale = 1000
    step = 0.1
    for alpha in range(0, int((1+step)*scale), int(step*scale)):
        alpha /= scale
        L_alpha = alpha * p_hat ** gamma_star + (1-alpha) *p_hat ** (gamma_star-2)
        p_hat_alpha = L_alpha * p_hat
        # print('alpha:', alpha, np.linalg.norm(p_hat - p_hat_alpha), delta * S_p)
        if np.linalg.norm(p_hat - p_hat_alpha) >= delta * S_p:
            alpha_star = alpha - step
            break

    L_alpha_star = alpha_star * p_hat ** gamma_star + (1-alpha_star) *p_hat ** (gamma_star-2)
    p_hat_nice = L_alpha_star * p_hat

    V_hat = np.diagflat(np.std(c, axis=0))
    P_hat_nice = np.matmul(np.matmul(V_hat, p_hat_nice), V_hat) # Covariance estimate
    return P_hat_nice

In [28]:
num_concepts = 500
batch_size = 64
c_hat, mu_hat, true_sigma_hat = sample_c(num_concepts, batch_size)
get_std = np.vectorize(get_corr_to_std(batch_size))


sigma_hat = get_cov(c_hat, get_std)
np_sigma = np.cov(c_hat, rowvar=False)

print(np.mean(np.absolute(true_sigma_hat-sigma_hat)), np.mean(np.absolute(true_sigma_hat)), np.mean(np.absolute(sigma_hat)))
print(np.mean(np.absolute(true_sigma_hat-np_sigma)), np.mean(np.absolute(true_sigma_hat)), np.mean(np.absolute(np_sigma)))

print(np.linalg.slogdet(true_sigma_hat))
print(np.linalg.slogdet(sigma_hat))
print(np.linalg.slogdet(np_sigma))


0.1719956398010254
262.90954629192595 627.5653749313013 364.6559352434739
235.4340234613548 627.5653749313013 392.1318918107918
SlogdetResult(sign=1.0, logabsdet=2182.2799581685376)
SlogdetResult(sign=1.0, logabsdet=1605.6184721785673)
SlogdetResult(sign=-1.0, logabsdet=-11220.156101056447)


In [38]:
def get_corr_to_std_torch(batch_size, std=1, mean=0, samples=100):
    min_corr = -1
    max_corr = 1
    step = 0.01
    scale_log = 2
    scale = 10**scale_log
    corr_to_std = []
    remapping_index = torch.arange(int(scale*min_corr), int((max_corr+step)*scale), int(step*scale))

    for corr in range(int(scale*min_corr), int((max_corr+step)*scale), int(step*scale)):
        corr /= scale
        cov_matrix = np.ones((2,2))
        np.fill_diagonal(cov_matrix, std**2)
        cov_matrix[0,1] = corr*std**2
        cov_matrix[1,0] = corr*std**2
        z = np.random.multivariate_normal(np.ones(2)*mean, cov_matrix, (samples, batch_size))
        corr_z = []
        for sample in z:
            corr_z.append(np.corrcoef(sample, rowvar=False)[0,1])
        std_corr_z = np.std(corr_z)
        corr_to_std.append(std_corr_z)
        
    corr_to_std = torch.tensor(corr_to_std)

    def get_std(correlation): # interpolation
        index = torch.round(correlation*scale)
        bucket_index = torch.bucketize(index.ravel(), remapping_index)
        return corr_to_std[bucket_index].reshape(bucket_index.shape)
    return get_std

In [43]:
def get_cov_torch(c, get_std, delta=1):
    p_hat = torch.corrcoef(c.T) # 256+128, 256+128

    # calculate S_p
    # start = time.time()
    # p_hat.cpu().detach().numpy()
    # print('time to detach', time.time()-start)
    start = time.time()
    S_p = torch.sum(get_std(p_hat)**2)**0.5
    print('time to convert:', time.time()-start)


    # estimate gamma
    for gamma in range(2, 10, 2):
        p_hat_gamma = p_hat**gamma * p_hat
        # print('gamma:', gamma, torch.linalg.norm(p_hat - p_hat_gamma), delta * S_p)
        if torch.linalg.norm(p_hat - p_hat_gamma) >= delta * S_p:
            gamma_star = gamma
            break

    # estimate alpha
    scale = 1000
    step = 0.1
    for alpha in range(0, int((1+step)*scale), int(step*scale)):
        alpha /= scale
        L_alpha = alpha * p_hat ** gamma_star + (1-alpha) *p_hat ** (gamma_star-2)
        p_hat_alpha = L_alpha * p_hat
        # print('alpha:', alpha, torch.linalg.norm(p_hat - p_hat_alpha), delta * S_p)
        if torch.linalg.norm(p_hat - p_hat_alpha) >= delta * S_p:
            alpha_star = alpha - step
            break

    L_alpha_star = alpha_star * p_hat ** gamma_star + (1-alpha_star) *p_hat ** (gamma_star-2)
    p_hat_nice = L_alpha_star * p_hat

    V_hat = torch.diagflat(torch.std(c, axis=0))
    P_hat_nice = torch.matmul(torch.matmul(V_hat, p_hat_nice), V_hat) # Covariance estimate
    return P_hat_nice

In [44]:
num_concepts = 500
batch_size = 64
c_hat, mu_hat, true_sigma_hat = sample_c(num_concepts, batch_size)
true_sigma_hat = torch.tensor(true_sigma_hat)
c_hat = torch.tensor(c_hat)
get_std = get_corr_to_std_torch(batch_size)


sigma_hat = get_cov_torch(c_hat, get_std)
torch_sigma = torch.cov(c_hat.T)

print(torch.mean(torch.absolute(true_sigma_hat-sigma_hat)), torch.mean(torch.absolute(true_sigma_hat)), torch.mean(torch.absolute(sigma_hat)))
print(torch.mean(torch.absolute(true_sigma_hat-torch_sigma)), torch.mean(torch.absolute(true_sigma_hat)), torch.mean(torch.absolute(torch_sigma)))

print(torch.logdet(true_sigma_hat))
print(torch.logdet(sigma_hat))
print(torch.logdet(torch_sigma))


time to convert: 0.001825571060180664
tensor(55.6570, dtype=torch.float64) tensor(625.3569, dtype=torch.float64) tensor(632.4664, dtype=torch.float64)
tensor(61.7789, dtype=torch.float64) tensor(625.3569, dtype=torch.float64) tensor(659.7617, dtype=torch.float64)
tensor(2173.3009, dtype=torch.float64)
tensor(1500.5106, dtype=torch.float64)
tensor(nan, dtype=torch.float64)


In [45]:
start = time.time()
torch_sigma = torch.cov(c_hat.T)
print(time.time()-start)

0.0017764568328857422
